# Simple NN 구현

In [79]:
import numpy as np 

In [41]:
applePie = np.array([1,0,0])
bugger = np.array([0,1,0])
chicken = np.array([0,0,1])

In [42]:
sunnyDay = np.array([1,0])
rainyDay = np.array([0,1])

화창한 SunnyDay 일떄는 ApplePie 를 ,
비가 오는날엔 Bugger를 먹는다.

In [43]:
matrix = np.array([[1,0],[0,1],[0,0]])

In [44]:
np.dot(matrix,sunnyDay.T)
# SunnyDay 니깐 ApplePie를 먹는다.

array([1, 0, 0])

# simple RNN 구현하기

In [124]:
food_matrix = np.array([[[1,0,0],[0,1,0],[0,0,1]],[[0,0,1],[1,0,0],[0,1,0]]])
weather_matrix = np.array([[[1,0],[1,0],[1,0]],[[0,1],[0,1],[0,1]]])

In [125]:
def makeonehot(c) :
    # [[0,1,0],[1,2,1]] 를 [[0,0,0],[0,1,0]] 과 같이 인코딩 하려면 어떻게 해야 할까..?
    
    for i in range(len(c)) :
        if(c[i] !=2) :
            c[i] = 0
        else :
            c[i] = 1
    return c




In [126]:
h_t = applePie
x_t = rainyDay

foodV = np.dot(food_matrix,h_t)
weatherV = np.dot(weather_matrix,x_t)
food_weather = np.concatenate((foodV+weatherV))
oneHot_food2weather = makeonehot(food_weather).reshape(2,3)

In [127]:
h_t = oneHot_food2weather[0]+oneHot_food2weather[1]

In [128]:
h_t

array([0, 1, 0])

In [135]:
x_t = np.array([[0,1],[1,0],[1,0],[0,1],[1,0],[0,1]]) # 흐림, 맑음, 맑음, 흐림, 맑음, 흐림
h_t = np.array([[1,0,0]])

def simpleRNN (x_t,h_t) :
    ht = h_t
    for i in range(len(x_t)) :
        foodV = np.dot(food_matrix,ht[i])
        weatherV = np.dot(weather_matrix,x_t[i])
        food_weather = np.concatenate((foodV+weatherV))
        oneHot_food2weather = makeonehot(food_weather).reshape(2,3)
        hap = oneHot_food2weather[0]+oneHot_food2weather[1]
        ht = np.append(ht,[hap], axis=0)
         
    return ht

print(simpleRNN(x_t,h_t))

[[1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]]


# VanilaRNN 

In [138]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, RNN
from keras import backend as K


In [139]:
# from https://keras.io/layers/recurrent/

class MinimalRNNCell(keras.layers.Layer):

    def __init__(self, units, **kwargs):
        self.units = units
        self.state_size = units
        super(MinimalRNNCell, self).__init__(**kwargs)

    def build(self, input_shape):
        # Whx 
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer='uniform',
                                      name='kernel')
        # Whh
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='uniform',
            name='recurrent_kernel')
        # bh
        self.bias = self.add_weight(
            shape=(1, self.units),
            initializer='uniform',
            name='bias')
        self.built = True

    def call(self, inputs, states):
        prev_output = states[0]
        h = K.dot(inputs, self.kernel)
        output = h + K.dot(prev_output, self.recurrent_kernel) + self.bias
#         output = keras.activations.sigmoid(output)
        return output, [output]

In [141]:
x = keras.Input((None, 5))
print(x)

Tensor("input_2:0", shape=(?, ?, 5), dtype=float32)


In [142]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6    # matlab 차트의 기본 크기를 15,6으로 지정해 줍니다.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [143]:
# csv 포맷의 데이터 경로를 지정합니다.
dataset_filepath = 'datasets/airline-passengers.csv'
# csv 파일을 읽어들여 DataFrame 객체를 생성합니다.
df = pd.read_csv(dataset_filepath)

FileNotFoundError: [Errno 2] File b'datasets/airline-passengers.csv' does not exist: b'datasets/airline-passengers.csv'

In [ ]:
train_data = [([df.iat[i+j,1] for j in range(5)], df.iat[i+5,1]) for i in range(115)]
test_data = [([df.iat[i+j,1] for j in range(5)], df.iat[i+5,1]) for i in range(115, 139)]
X_train = np.array([X for (X,y) in train_data])
X_test = np.array([X for (X,y) in test_data])
y_train = np.array([y for (X,y) in train_data])
y_test = np.array([y for (X,y) in test_data])
X_val = X_test
y_val = y_test

In [ ]:
# RNN의 경우 Dense와 입출력 tensor shape가 다르다.
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_val = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# to reproduce same training result
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [ ]:
cell = MinimalRNNCell(50)
# cell.build((5, 1))
rnn_layer = RNN(cell)
# rnn_layer.build((5, 1))

# define model
model = Sequential()
#model.add(Dense(100, activation='relu', input_dim=5))   
model.add(rnn_layer)  # Dense Layer 대신 RNN Layer를 사용
model.add(Dense(1))
model.build((None, 5, 1))   # model의 input shape를 지정해 주어야 summary가 가능하다.
model.summary()

model.compile(optimizer='adam', loss='mse')

In [ ]:
# fit model
model.fit(X_train, y_train, epochs=1000, verbose=1, validation_data=(X_val, y_val))

In [ ]:

# check how exactly fit train data

y_hats = []
cur_y_hat = 0
for i in range(115):
    x_input = X_train[i]
    x_input_reshape = x_input.reshape(1,5,1)
    cur_y_hat = model.predict(x_input_reshape, verbose=0)
    cur_y_hat = np.squeeze(cur_y_hat)
    print('TRAIN DATA %d : predicted=%f, ground_truth=%d' % (i, cur_y_hat, y_train[i]))
    y_hats.append(cur_y_hat)

In [ ]:

plt.plot(y_train)
plt.plot(y_hats)
plt.title('RMSE: %.4f'% np.sqrt(sum((y_hats-y_train)**2)/len(y_train)))

In [ ]:

# predict with trained model

y_hats = []
cur_y_hat = 0
for i in range(24):
    if i == 0:  # test data 의 맨 첫번째 row에 대해서만 X 값을 그대로 활용
        x_input = X_test[0]
    else:       # 두번째 row부터는 이전 row의 prediction을 반영하여 X를 재구성
        x_input = x_input.tolist()
        del x_input[0]
        x_input.append(cur_y_hat)
        x_input = np.array(x_input)
    x_input_reshape = x_input.reshape(1,5,1)
    cur_y_hat = model.predict(x_input_reshape, verbose=0)
    cur_y_hat = np.squeeze(cur_y_hat)
    print('TEST DATA %d : predicted=%f, ground_truth=%d' % (i, cur_y_hat, y_test[i]))
    y_hats.append(cur_y_hat)

In [ ]:
plt.plot(y_test)
plt.plot(y_hats)
plt.title('RMSE: %.4f'% np.sqrt(sum((y_hats-y_test)**2)/len(y_test)))